In [ ]:
import gpytorch
import torch
from LowPrecisionApproxGP import KERNEL_FACTORY, DATASET_FACTORY
from LowPrecisionApproxGP.util.GreedyTrain import greedy_train
from LowPrecisionApproxGP.model.models import VarPrecisionModel
from LowPrecisionApproxGP import load_bikes, load_road3d, load_energy, load_naval, load_protein

In [ ]:
# # This is for baseline model, Vanilla Exact GP Model
# class ExactGPModel(gpytorch.models.ExactGP):
#     def __init__(self, train_x, train_y, likelihood):
#         super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
#         self.mean_module = gpytorch.means.ConstantMean()
#         self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

#     def forward(self, x):
#         mean_x = self.mean_module(x)
#         covar_x = self.covar_module(x)
#         return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
# Params to tweak when running experiments
dtype = torch.float32
kwargs = {
    "training_iter": 50,
    "max_inducing_points": 50,
    "Model Name" : "JupyterNotebookModel",
    "dtype":dtype,
    "use_max" : True,
    "j" : 10,
    "max_js": 5,
}

In [ ]:
# Load bikes dataset as dtype / tensor
train_data, test_data = load_bikes(dtype) # Make sure you download data first
train_x, train_y = train_data
test_x, test_y = test_data

# Create Likelihood / Model
base_kernel = KERNEL_FACTORY["base"] # Just Vanilla Base Kernel (Scale * RBF kernel)
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = VarPrecisionModel(train_x, train_y, likelihood, dtype, gpytorch.means.ConstantMean(), base_kernel())
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood,model)

# Set to training mode
model.train()
likelihood.train()


In [ ]:
# Train
greedy_train(
        train_data=(train_x, train_y),
        model=model,
        mll=mll,
        max_iter=kwargs.get("training_iter"),
        max_inducing_points=kwargs.get("max_inducing_points"),
        model_name=kwargs.get("Model Name"),
        dtype=kwargs.get("dtype"),
        use_max=kwargs.get("use_max"),
        j=kwargs.get("j"),
        max_js=kwargs.get("max_js"),
    )

In [ ]:
# Eval Performance
model.eval()
likelihood.eval()

with torch.no_grad():
    trained_pred_dist = likelihood(model(test_x))
    predictive_mean = trained_pred_dist.mean
    lower, upper = trained_pred_dist.confidence_region()

final_msll = gpytorch.metrics.mean_standardized_log_loss(trained_pred_dist, test_y)
final_mse = gpytorch.metrics.mean_squared_error(
    trained_pred_dist, test_y, squared=True
)

final_mae = gpytorch.metrics.mean_absolute_error(trained_pred_dist, test_y)
print(f"Mean_Standardized_Log_Test_Loss:{final_msll}")
print(f"Mean_Squared_Test_Error:{final_mse}")
print(f"Mean_Absolute_Test_Error:{final_mae}")